# Visualizing the GAN results

Let's visualize some random results from the GAN. To do so we just need to load a generator model and feed it with random Gaussian noise of the approriate size

In [ ]:
import os
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F

import torchvision as tv
from torchvision.transforms import v2
import torchvision.transforms.functional as TF

# Get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

## Visualizing the results in a notebook

The variable `LATENT_DIM` defines the dimension of your ["latent vector"](https://medium.com/@jain.yasha/gan-latent-space-1b32cd34cfda). If you changed the same variable in the training notebook, you will have to change it here as well.

The third, `iter` defines the epoch for which you want to load a model. You can examine the directory and the example images for each epoch, to choose which epoch you want to visualize.

In [ ]:
LATENT_DIM = 100 # The size of the latent space/input vector

N_CHANNELS = 1 # 3 for colour
IMAGE_SHAPE = (N_CHANNELS,64,64) # C, H, W

G_DIM = 64
D_DIM = 64

# fixed directory structure -------------
DATASETS_DIR = pathlib.Path("datasets")
DATASETS_DIR.mkdir(exist_ok=True)

MODELS_DIR = pathlib.Path("models")
MODELS_DIR.mkdir(exist_ok=True)

GENERATED_DIR = pathlib.Path("generated")
GENERATED_DIR.mkdir(exist_ok=True)
# ----------------------------------------

MODEL_NAME = "dcgan_mnist" # change accordingly

DCGAN_DIR = MODELS_DIR / MODEL_NAME

ITERS = 936 # change if needed
GENERATOR_NAME = f"{MODEL_NAME}_g.iter_{ITERS:04d}.pt"

# generated images
GENERATED_DIR = GENERATED_DIR / f"{MODEL_NAME}_images"
GENERATED_DIR.mkdir(exist_ok=True)

We must redefine the net, then load the weights.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim, n_channels):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            #                  input, output, kernel, stride, padding
            nn.ConvTranspose2d(latent_dim, output_dim * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(G_DIM * 8),
            nn.ReLU(True),
            # state size. (G_DIM*8) x 4 x 4
            nn.ConvTranspose2d(output_dim * 8, output_dim * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(output_dim * 4),
            nn.ReLU(True),
            # state size. (output_dim*4) x 8 x 8
            nn.ConvTranspose2d(output_dim * 4, output_dim * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(output_dim * 2),
            nn.ReLU(True),
            # state size. (output_dim*2) x 16 x 16
            nn.ConvTranspose2d(output_dim * 2, output_dim, 4, 2, 1, bias=False),
            nn.BatchNorm2d(output_dim),
            nn.ReLU(True),
            # state size. (output_dim) x 32 x 32
            nn.ConvTranspose2d(output_dim, n_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (n_channels) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

G = Generator(
    latent_dim=LATENT_DIM,
    output_dim=G_DIM,
    n_channels=N_CHANNELS
).to(device)

# reloading
G.load_state_dict(
    torch.load(
        DCGAN_DIR / GENERATOR_NAME,
        map_location=torch.device(device)
    )
)

print(G)
print()
print(f"Our model has {sum(p.numel() for p in G.parameters()):,} parameters.")

In [ ]:
# utils

def denorm(x):
    """Denormalize the outputs from [-1, 1] to [0,1] (generator with 'tanh' activation)"""
    return (x * 0.5) + 0.5

In [ ]:
## Grid generation

In [ ]:
N_IMAGES = 64
fixed_noise = torch.randn(N_IMAGES, LATENT_DIM, 1, 1, device=device)

def make_grid(noise, iters=0, figsize=(8,8), show=True, save=False):
    with torch.no_grad():
        output = G(noise).cpu().detach()    
    plt.figure(figsize=figsize)
    plt.axis("off")
    plt.imshow(
        TF.to_pil_image(
            tv.utils.make_grid(output, padding=2, normalize=True).cpu()
        )
    )
    if save:
        plt.savefig(GENERATED_DIR / f"single_image.iter_{iters}_{i:04d}.png")
    if show:
        plt.show()

make_grid(fixed_noise)

## Single image generation

In [ ]:
N_IMAGES = 3
fixed_noise = torch.randn(N_IMAGES, LATENT_DIM, 1, 1, device=device)

def make_images(noise, iters=0, figsize=(6,6), show=True, save=False):
    with torch.no_grad():
        output = G(noise).cpu().detach()
    for i, o in enumerate(output):
        img = TF.to_pil_image(denorm(o))
        plt.figure(figsize=figsize)
        plt.axis("off")
        plt.imshow(img, cmap='gray')
        if save:
            plt.savefig(GENERATED_DIR / f"single_image.iter_{iters}_{i:04d}.png")
        if show:
            plt.show()

make_images(fixed_noise)

## Random Walk

We can use a loop to gradually add some random noise to our latent vector, effectively 'moving' (blindly, chaotically) in the latent space.

In [ ]:
seed = torch.randn(1, 100,1,1)
noise = torch.randn(seed.size()) * .2
noise = torch.rand(seed.size()) * .2
noise.min(), noise.max(), noise.mean()

In [ ]:
def generate_random_walk(generator, latent_dim, n=64, noise_mode="normal"):
    seed = torch.randn(1, latent_dim,1,1)
    random_latent_vectors = [seed]
    for t in range(1, n):
        if noise_mode == "normal":
            # `randn` yields normally distributed numbers (mean 0, std 1)
            # -> for an std of .2, we multiply by it (can be tweaked!
            noise = torch.randn(seed.size()) * .2
        if noise_mode == "uniform":
            # Uniform Noise, between 0 and 1, we shift that by .5:
            # try tweaking the min/max values!
            noise =  torch.rand(seed.size()) * .2 - .5
        # increment our vector
        random_latent_vectors.append(random_latent_vectors[-1] + noise)
     # stack the tensors along the batch dim (0) and move to device
    return torch.cat(random_latent_vectors, dim=0).to(device)

make_grid(generate_random_walk(G, 100))

In [ ]:
make_grid(generate_random_walk(G, 100, noise_mode="uniform"))

In [ ]:
make_images(generate_random_walk(G, 100, n=4))

## Interpolating in latent space

We can interpolate between one point in the latent space (the variable `a`) and another point (the variable `b`) to produce a smooth transition between images generated by the GAN along the latent space. It is recommended to use a geod "spherical linear interpolation", which effectively describes a ["geodesic"](https://en.wikipedia.org/wiki/Geodesic) ([mini-vid](https://www.youtube.com/watch?v=KsdIuVByfMc)). We use spherical interpolation because the multivariate Gaussian used as an input to the GAN generator can be approximated by a hypersphere (a sphere in high dimensions).

See [this discussion](https://github.com/soumith/dcgan.torch/issues/14) and [this post](https://machinelearningmastery.com/how-to-interpolate-and-perform-vector-arithmetic-with-faces-using-a-generative-adversarial-network/) for technical details and to see where the interpolation code comes from.

In [ ]:
from numpy.linalg import norm

def lerp(t, a, b):
    return a + t*(b - a)

def slerp(val, low, high):
    omega = np.arccos(np.clip(np.dot(low/norm(low), high/norm(high)), -1.0, 1.0))
    so = np.sin(omega)
    if so == 0:
        # L'Hopital's rule/LERP
        return (1.0-val) * low + val * high
    return np.sin((1.0-val)*omega) / so * low + np.sin(val*omega) / so * high

In [ ]:
seed1 = torch.randn(1, 100,1,1)
seed2 = torch.randn(1, 100,1,1)
b = seed1 @ seed2
# c = torch.sum(seed1 * seed2, dim=-1)
d = (seed1 @ seed2.transpose(-2, -1))

In [ ]:
def slerp(val, low, high):

    # Compute the cosine of the angle between the vectors and clip 
    # it to avoid out-of-bounds errors
    omega = torch.acos(torch.clamp(low @ high, -1.0, 1.0))
    so = torch.sin(omega)

    return torch.where(
        so == 0,
        # If sin(omega) is 0, use LERP (linear interpolation)
        (1.0 - val) * low + val * high,
        # Otherwise perform spherical interpolation (SLERP)
        (torch.sin((1.0 - val) * omega) / so) * low + (torch.sin(val * omega) / so) * high
    )


In [ ]:
def generate_interpolated(generator, latent_dim, n=64):
    # here we create to random vectors: one other way might be 
    # to create one, then add a tiny bit of noise to it, then
    # interpolate between the two?
    seed1 = torch.randn(1, latent_dim,1,1)
    seed2 = torch.randn(1, latent_dim,1,1)
    random_latent_vectors = []
    for t in np.linspace(0, 1, n):
        random_latent_vectors.append(slerp(t,seed1, seed2))
     # stack the tensors along the batch dim (0) and move to device
    return torch.cat(random_latent_vectors, dim=0).to(device)

make_grid(generate_interpolated(G, 100))

In [ ]:
make_images(generate_interpolated(G, 100, n=10))

## Experiments

Importing the tools from the training notebook, it is possible to create videos using these!

There are other, advanced techniques that people have explored here's, a tutorial with a few ideas (in TensorFlow): [Generate Artificial Faces with CelebA Progressive GAN Model](https://www.tensorflow.org/hub/tutorials/tf_hub_generative_image_module).